In [6]:
import pandas as pd
import sqlite3

# 1. Connect to the database
conn = sqlite3.connect("annotations.db")

# 2. (Optional) List all available tables to check what's inside
annotations = pd.read_sql_query("SELECT * FROM annotations;", conn)
print(annotations)

# 3. Read a specific table (replace 'your_table_name' with the actual name)
demographics = pd.read_sql_query("SELECT * FROM demographics;", conn)
print(demographics)


# 4. Close the connection
conn.close()

demographics.columns




      annotation_id               prolific_id                session_id  \
0                 1                    amanda                      test   
1                 2                    amanda                      test   
2                 3                    amanda                      test   
3                 4                    amanda                      test   
4                 5                    amanda                      test   
...             ...                       ...                       ...   
1417           1418  67e2b6561ccb8f5bb055cc85  68e78b5d72c8b85ac2049fc3   
1418           1419  67e2b6561ccb8f5bb055cc85  68e78b5d72c8b85ac2049fc3   
1419           1420  67e2b6561ccb8f5bb055cc85  68e78b5d72c8b85ac2049fc3   
1420           1421  67e2b6561ccb8f5bb055cc85  68e78b5d72c8b85ac2049fc3   
1421           1422  67e2b6561ccb8f5bb055cc85  68e78b5d72c8b85ac2049fc3   

                              candidate_id knowledge power status trust  \
0     a389a7e3-bec5-47cf

Index(['prolific_id', 'session_id', 'gender', 'gender_other', 'age',
       'nationality', 'ethnicity', 'ethn_free', 'language', 'religion',
       'education', 'big_1', 'big_2', 'big_3', 'big_4', 'big_5', 'big_6',
       'big_7', 'big_8', 'big_9', 'big_10'],
      dtype='object')

In [ ]:
column_names = ['prolific_id', 'session_id', 'gender', 'gender_other', 'age',
       'nationality', 'ethnicity', 'ethn_free', 'language', 'religion',
       'education', 'big_1', 'big_2', 'big_3', 'big_4', 'big_5', 'big_6',
       'big_7', 'big_8', 'big_9', 'big_10']
demographics = pd.read_csv('pilot 3 annotations/Demographics - Sheet1.tsv',header=None, names=column_names, sep = '\t')
annotations = pd.read_csv('pilot 3 annotations/Annotations - Sheet1.tsv', sep = '\t')
demographics.columns


Index(['test', 'test.1', 'Female', 'Unnamed: 3', '55-60', 'United Kingdom',
       'Asian / Pacific Islander', 'Unnamed: 7', 'Arabic', 'Buddhist',
       'High school or below', 'Neutral', 'Neutral.1', 'Neutral.2', 'Agree',
       'Agree.1', 'Neutral.3', 'Neutral.4', 'Neutral.5', 'Neutral.6',
       'Agree.2'],
      dtype='object')